This file, once complete, will obtain a dataframe that contains every trade that occured during the 2022-23 NHL season. I am going to scrape the CapFriendly Website using beautiful soup and selenium and then use pandas to get it into the dataframe.

In [1]:
import requests
import pandas as pd
import time
import logging
import lxml
from selenium import webdriver
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

Now that necessary packages are installed, going to parse the needed url. This URL has a dynamic table that contains the cost per save of NHL Goalies for the 2016-17 season. So I am going to scrape the cap friendly website (Cost Per Save tab) and get this into a dataframe that I can eventually download and put into the MySQL database. As mentioned, since the table is dynamic and not static, the selenium package is needed because the table only expands to its full length once you scroll down.

In [2]:
# Set up the Selenium driver (make sure you have the correct path to your WebDriver)
driver_path = 'C:/STL Blues Analytics/SalaryCap/NHLSalaryCap/chromedriver.exe'  # Update this to your WebDriver path
service = Service(driver_path)
driver = webdriver.Chrome(service=service)

# Open the webpage
url = "https://www.capfriendly.com/cost-per-save/2017/season"
driver.get(url)

# Get initial page height
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # Scroll down to the bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    # Wait for new content to load
    time.sleep(2)
    
    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    
    if new_height == last_height:
        break
    last_height = new_height

# Get the page source after all rows are loaded
page_source = driver.page_source
driver.quit()

# Parse the page source with BeautifulSoup
soup = BeautifulSoup(page_source, 'html.parser')

# Find the table and convert it to a DataFrame
table = soup.find('table')
dfattempt = pd.read_html(str(table))[0]



C:\Users\nsofi\AppData\Local\Temp\ipykernel_10748\436904512.py:36: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  dfattempt = pd.read_html(str(table))[0]


In [3]:
dfattempt.shape
dfattempt.tail(5)

,#,GOALIE,AGE,TYPE,EXPIRY,CAP HIT,P. BONUSES,GP,MIN,SH,SV,SV%,GAA,$/GP,$/MIN,$/SV
67,68,"Varlamov, Semyon",28,NaN,Stnd,UFA,"$5,900,000",24,1347,745,669.0,0.898,3.39,"$245,833","$4,380","$8,819"
68,69,"Berra, Reto",29,NaN,Stnd,UFA,"$1,450,000",7,313,145,127.0,0.876,3.45,"$207,142","$4,632","$11,417"
69,70,"Quick, Jonathan",30,NaN,Stnd,UFA,"$5,800,000",17,930,421,386.0,0.917,2.26,"$341,176","$6,236","$15,025"
70,71,"Hammond, Andrew",28,NaN,Stnd,UFA,"$1,350,000",6,205,86,72.0,0.837,4.10,"$225,000","$6,585","$18,750"
71,72,"Pavelec, Ondrej",29,NaN,Stnd,UFA,"$3,900,000",8,439,233,207.0,0.888,3.55,"$487,500","$8,883","$18,840"


In [5]:
dfattempt.to_csv("CF Cost Per Save 2016-17.csv") 